## ECDS emergency care admission vs SUS hospital admission validation

There is a delay between patients being admitted to hospital with COVID-19 and this data being made available.  SUS is the 'gold standard' for COVID-19 hospital admission but is coded 3 weeks after completion of an episode.  This means we cannot rapidly evaluate vaccine effectiveness with respect to reducing hospital admission.

As a large proportion of hospital admission comes through A&E attendance, emergency admission data through ECDS may provide a good enough picture of hospital admission.  Below is a validation of this.

### Methods

* Patients hospitalised due to COVID-19 are first identified using SUS.
* Patients admitted to emergency care due to COVID-19 are identified using the criteria below:
    * All patients admitted to A&E AND then discharged to hospital or ICU are selected.
    * In these patients, those admitted due to COVID-19 are filtered to include those that
        * Are admitted with positive COVID-19 code OR
        * Have had positive COVID-19 test in the 2 weeks prior to admission OR
        * Have been recorded as COVID-19 positive in primary care 2 weeks prior to admission (primary care positive test, primary care covid code, primary care covid sequalae). 
* Ability of ECDS data to identify those admitted to hospital due to COVID-19 vs SUS data is assessed.


In [114]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.0f}'.format

In [129]:
df = pd.read_csv('../output/input.csv')
ae_discharge_dict = {"discharged_to_ward": 306706006, "discharged_to_icy": 1066391000000105}
ae_discharge_list = [value for (key, value) in ae_discharge_dict.items()]


positive_covid_patients_sus = df[df['covid_hospital_admission'].notna()]
negative_covid_patients_sus = df[~df['covid_hospital_admission'].notna()]

positive_covid_patients_ecds = df[((df['attended_ae']==1) & (df['discharge_destination'].isin(ae_discharge_list))) & ((df['attended_ae_covid_status']==1) | (df['positive_covid_test_before_admission'] ==1) | (df['covid_primary_care_before_admission'] ==1))]
negative_covid_patients_ecds = df[~((df['attended_ae']==1) & (df['discharge_destination'].isin(ae_discharge_list))) & ((df['attended_ae_covid_status']==1) | (df['positive_covid_test_before_admission'] ==1) | (df['covid_primary_care_before_admission'] ==1))]


sus_patients_positive = list(positive_covid_patients_sus['patient_id'])
ecds_patients_positive = list(positive_covid_patients_ecds['patient_id'])
sus_patients_negative = list(negative_covid_patients_sus['patient_id'])
ecds_patients_negative = list(negative_covid_patients_ecds['patient_id'])

In [131]:
#sensitivity - number of sus identified by ecds
#specificity - number of those not in sus who are not in ecds

sensitivity = len(list(set(sus_patients_positive).intersection(ecds_patients_positive)))/len(sus_patients_positive)
print(f"Sensitivity: {sensitivity}")

specificity = len(list(set(sus_patients_negative).intersection(ecds_patients_negative)))/len(sus_patients_negative)
print(f"Specificity : {specificity}")


Sensitivity: 0.15384615384615385
Specificity : 0.78542104736157
